In [1]:
import simulate
import baseline
import dataload
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#import pypots
from pypots.data import load_specific_dataset, mcar, masked_fill
from pypots.imputation import SAITS, BRITS
from pypots.utils.metrics import cal_mae, cal_rmse, cal_mre

# UCI HAR

In [2]:
dict = dataload.uci()
X = dict["X"]
y = dict["y"]

In [3]:
# MCAR
dict_MCAR = simulate.simulate_nan(X, 0.1, "MCAR")
X_intact_MCAR = dict_MCAR['X_init']
X_missing_MCAR = dict_MCAR['X_incomp']
X_mask_MCAR = dict_MCAR['mask']


In [4]:
import importlib
import simulate #import the module here, so that it can be reloaded.
importlib.reload(simulate)


# MAR - logistic
dict_MAR = simulate.simulate_nan(X, 0.1, "MAR")
X_intact_MAR = dict_MAR['X_init']
X_missing_MAR = dict_MAR['X_incomp']
X_mask_MAR = dict_MAR['mask']

In [5]:
import simulate #import the module here, so that it can be reloaded.
importlib.reload(simulate)

# MNAR - logistic
dict_MAR = simulate.simulate_nan(X, 0.1, "MNAR", opt="logistic")
X_intact_MNAR = dict_MAR['X_init']
X_missing_MNAR = dict_MAR['X_incomp']
X_mask_MNAR = dict_MAR['mask']

X_intact = [X_intact_MCAR, X_intact_MAR, X_intact_MNAR]
X_missing = [X_missing_MCAR, X_missing_MAR, X_missing_MNAR]
X_mask = [X_mask_MCAR, X_mask_MAR, X_mask_MNAR]
missingness = ["MCAR", "MAR", "MNAR"]

In [6]:
print(np.unique(X_mask_MAR, return_counts=True))
print(np.unique(X_mask_MCAR, return_counts=True))
print(np.unique(X_mask_MNAR, return_counts=True))

(array([0., 1.]), array([10677991,  1186457], dtype=int64))
(array([0., 1.]), array([10677841,  1186607], dtype=int64))
(array([0., 1.]), array([10679971,  1184477], dtype=int64))


In [7]:
result_uci_imputation = []

In [8]:
# Zero-filling imputation
res = []
for i in range(3):
    X_zero_imputed = baseline.zero_filling_imputation(X_missing[i])
    res.append(cal_mae(X_intact[i], X_zero_imputed, X_mask[i]))
    res.append(cal_rmse(X_intact[i], X_zero_imputed, X_mask[i]))
    res.append(cal_mre(X_intact[i], X_zero_imputed, X_mask[i]))

result_uci_imputation.append(res)

In [9]:
# Mean imputation
res = []

for i in range(3):
    X_mean_imputed = baseline.mean_imputation(X_missing[i])
    res.append(cal_mae(X_intact[i], X_mean_imputed, X_mask[i]))
    res.append(cal_rmse(X_intact[i], X_mean_imputed, X_mask[i]))
    res.append(cal_mre(X_intact[i], X_mean_imputed, X_mask[i]))

result_uci_imputation.append(res)

In [10]:
# Median imputation
res = []

for i in range(3):
    X_median_imputed = baseline.median_imputation(X_missing[i])
    res.append(cal_mae(X_intact[i], X_median_imputed, X_mask[i]))
    res.append(cal_rmse(X_intact[i], X_median_imputed, X_mask[i]))
    res.append(cal_mre(X_intact[i], X_median_imputed, X_mask[i]))

result_uci_imputation.append(res)

In [11]:
# BRITS

res = []

for i in range(3):
    brits = BRITS(n_steps=128, n_features=9,rnn_hidden_size=1, epochs=3)
    brits.fit(X_missing[i])
    imputation = brits.impute(X_missing[i])
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_uci_imputation.append(res)

Model initialized successfully. Number of the trainable parameters: 926
epoch 0: training loss 0.3934
epoch 1: training loss 0.1853
epoch 2: training loss 0.1351
Finished training.
Model initialized successfully. Number of the trainable parameters: 926
epoch 0: training loss 0.3894
epoch 1: training loss 0.1776
epoch 2: training loss 0.1284
Finished training.
Model initialized successfully. Number of the trainable parameters: 926
epoch 0: training loss 0.4458
epoch 1: training loss 0.1935
epoch 2: training loss 0.1342
Finished training.


In [14]:
# SAITS

res = []

for i in range(3):
    saits = SAITS(n_steps=128, n_features=9, n_layers=2, d_model=256, d_inner=128, n_head=4, d_k=64, d_v=64, dropout=0.2, epochs=10)
    saits.fit(X_missing[i])
    imputation = saits.impute(X_missing[i])
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_uci_imputation.append(res)

Model initialized successfully. Number of the trainable parameters: 1332038
epoch 0: training loss 0.1932
epoch 1: training loss 0.1259
epoch 2: training loss 0.1133
epoch 3: training loss 0.1066
epoch 4: training loss 0.1024
epoch 5: training loss 0.0994
epoch 6: training loss 0.0970
epoch 7: training loss 0.0954
epoch 8: training loss 0.0939
epoch 9: training loss 0.0926
Finished training.
Model initialized successfully. Number of the trainable parameters: 1332038
epoch 0: training loss 0.2071
epoch 1: training loss 0.1266
epoch 2: training loss 0.1130
epoch 3: training loss 0.1064
epoch 4: training loss 0.1024
epoch 5: training loss 0.0992
epoch 6: training loss 0.0969
epoch 7: training loss 0.0951
epoch 8: training loss 0.0937
epoch 9: training loss 0.0927
Finished training.
Model initialized successfully. Number of the trainable parameters: 1332038
epoch 0: training loss 0.1914
epoch 1: training loss 0.0976
epoch 2: training loss 0.0786
epoch 3: training loss 0.0726
epoch 4: train

In [15]:
errors = ["mae", "rmse", "mre"]
cols = []

for i in missingness:
    for j in errors:
        cols.append(i+" "+j)

methods = ["zero-filling imputation", "mean imputation", "median imputation", "BRITS", "SAITS"]
result_uci_imputation = pd.DataFrame(result_uci_imputation, columns = cols, index=methods)

In [16]:
result_uci_imputation

,MCAR mae,MCAR rmse,MCAR mre,MAR mae,MAR rmse,MAR mre,MNAR mae,MNAR rmse,MNAR mre
zero-filling imputation,0.114760,0.233358,1.361754e+14,0.114956,0.233186,1.363906e+14,0.115120,0.233995,1.363567e+14
mean imputation,0.115665,0.232491,7.882437e+00,0.115825,0.232272,7.963740e+00,0.116015,0.233281,7.879241e+00
median imputation,0.113521,0.234953,5.544886e+00,0.113671,0.234654,5.504210e+00,0.113984,0.235801,5.471261e+00
BRITS,0.037630,0.095399,3.279052e-01,0.039161,0.100336,3.406561e-01,0.039876,0.098917,3.463862e-01
SAITS,0.036589,0.090022,3.188264e-01,0.038674,0.096277,3.364215e-01,0.034049,0.087465,2.957710e-01
